In [1]:
import ctypes
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mlp_model = ctypes.CDLL("modele/mlp/target/release/libmlp_classification.so")

# Définir les types d'arguments et de retour des fonctions C
mlp_model.mlpInit.argtypes = [ctypes.POINTER(ctypes.c_uint), ctypes.c_size_t, ctypes.c_double]
mlp_model.mlpInit.restype = ctypes.POINTER(ctypes.c_void_p)

mlp_model.mlpTrain.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    ctypes.POINTER(ctypes.c_double),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_size_t,
    ctypes.c_bool
]
mlp_model.mlpTrain.restype = None


mlp_model.mlpBackward.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    ctypes.POINTER(ctypes.c_double),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t,
    ctypes.c_bool
]
mlp_model.mlpBackward.restype = ctypes.c_double

mlp_model.mlpUpdateWeights.argtypes = [ctypes.POINTER(ctypes.c_void_p), ctypes.c_double]
mlp_model.mlpUpdateWeights.restype = None

mlp_model.mlpForward.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t,
    ctypes.c_bool
]
mlp_model.mlpForward.restype = ctypes.POINTER(ctypes.c_double)


mlp_model.mlpPredict.argtypes = [
    ctypes.POINTER(ctypes.c_void_p),
    ctypes.POINTER(ctypes.c_double),
    ctypes.c_size_t,
    ctypes.c_bool,
    ctypes.POINTER(ctypes.c_double)
]
mlp_model.mlpPredict.restype = None

mlp_model.mlpFree.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
mlp_model.mlpFree.restype = None


In [ ]:
parameter = np.array([2, 3, 1], dtype=np.uintp)
learning_rate = 0.01
epochs = 100000
parameter_ptr = parameter.ctypes.data_as(ctypes.POINTER(ctypes.c_uint))
nn = mlp_model.mlpInit(parameter_ptr, parameter.size, learning_rate)

X_train = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 0.0],
    [1.0, 1.0]
], dtype=np.float64)

y_train = np.array([
    [1.0, 0.0],
    [0.0, 1.0],
    [0.0, 1.0],
    [1.0, 0.0]
], dtype=np.float64)

In [ ]:
mlp_model.mlpTrain(
    nn,
    X_train.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    y_train.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    X_train.shape[0],
    X_train.shape[1],
    epochs,  # epochs
    False   # is_classification
)

In [ ]:
for k in range(len(X_train)):
    predictions = np.zeros(1, dtype=np.float64)
    mlp_model.mlpPredict(
        nn,
        X_train[k].ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
        X_train[k].size,
        True,
        predictions.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
    )
    print("Sample", X_train[k], ", predictions =", predictions)
